<a href="https://colab.research.google.com/github/BinBin264/fish_classification/blob/BE/Fish_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Fish Image Species Classification  



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

#  load species dataset

In [ ]:

dir = Path('../input/fish-species/Species/Test_Set')

filepaths = list(dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))


filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
dataframe2_test = pd.concat([filepaths , labels] , axis=1)
dataframe2_test


In [ ]:

dir = Path('../input/fish-species/Species/Training_Set')

filepaths = list(dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))


filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
dataframe2_train = pd.concat([filepaths , labels] , axis=1)
dataframe2_train

In [ ]:
dataframe2_train['Label'].value_counts()

In [ ]:
dataframe2_test['Label'].value_counts()

In [ ]:
dataframe2_test

In [ ]:
dataframe = pd.concat([dataframe2_train,dataframe2_test],axis =0 )

In [ ]:
dataframe['Label'].value_counts()

In [ ]:

samples = []
for category in dataframe['Label'].unique():
    category_slice = dataframe.query("Label == @category")
    samples.append(category_slice.sample(500, random_state=1))

dataframe = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)


In [ ]:
dataframe['Label'].value_counts()

# Creating train, test DataFrames

In [ ]:
train_df, test_df = train_test_split(dataframe, train_size=0.9, shuffle=True, random_state=1)

In [ ]:
train_df.shape

# Loading the Images for mobilenetv2


In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    height_shift_range=0.2,
    width_shift_range=0.2,
    rotation_range=40,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(160, 160),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=128,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(160, 160),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=128,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(160, 160),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=128,
    shuffle=False
)

In [ ]:
test_images.class_indices

# Load Pretrained MobileNetV2 Model

In [ ]:
import tensorflow as tf
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False


# Training

In [ ]:

from tensorflow.keras.layers import *
from tensorflow.keras.models import *



model = Sequential ([
                        pretrained_model,
                        Flatten(),
                        Dropout(0.2),
                        Dense(128, activation='relu'),
                        Dense(64, activation='relu'),
                        Dense(20, activation='softmax')
                    ])


model.summary()



In [ ]:

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Learning rate schedule
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Model checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model_mbn.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

In [ ]:
history = model.fit(train_images,
                    validation_data=val_images,
                    epochs=30,
                    callbacks=[early_stopping, checkpoint])

In [ ]:
model.save('last_mobile.keras')

# Results

In [ ]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))


import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])
plt.title('Training and Validation losses')
plt.xlabel('epoch')


plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Training', 'Validation'])
plt.title('Training and Validation binary_accuracy')
plt.xlabel('epoch')


In [ ]:
pred = model.predict(test_images)
pred = np.argmax(pred, axis=1)

In [ ]:
predict_data=test_df.copy()
labels={}
for l,v in test_images.class_indices.items():
    labels.update({v:l})
predict_data['pred']=pred
predict_data['pred']=predict_data['pred'].apply(lambda x: labels[x])

In [ ]:
predict_data=predict_data.reset_index(drop=True)
predict_data.head(10)
predict_data=predict_data.reset_index(drop=True)
predict_data.head(10)

In [ ]:
predict_data[predict_data['Label']!=predict_data['pred']]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
print(f"Accuracy Score: {accuracy_score(predict_data['Label'],predict_data['pred'])}")
plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(predict_data['Label'],predict_data['pred']), annot=True, fmt='2d')

In [ ]:
print(classification_report(predict_data['Label'],predict_data['pred']))

Fine tunning


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def mobilenet_like_model(input_shape=(224, 224, 3), num_classes=20):
    """
    Create a MobileNetV2-like model for classification

    Args:
        input_shape (tuple): Shape of input images
        num_classes (int): Number of output classes

    Returns:
        tf.keras.Model: Compiled neural network model
    """
    def _depthwise_conv_block(inputs, filters, kernel=3, strides=1, expansion=6, block_id=None):
        """
        Depthwise Separable Convolution Block

        Args:
            inputs (tensor): Input tensor
            filters (int): Number of output filters
            kernel (int): Kernel size for depthwise convolution
            strides (int): Strides for convolution
            expansion (int): Expansion factor for intermediate layer
            block_id (int): Identifier for the block

        Returns:
            tensor: Output tensor after convolution block
        """
        input_shape = tf.keras.backend.int_shape(inputs)

        # Expansion layer
        x = layers.Conv2D(
            int(input_shape[-1] * expansion),
            kernel_size=1,
            padding='same',
            use_bias=False,
            activation=None,
            name=f'block_{block_id}_expand'
        )(inputs)
        x = layers.BatchNormalization(name=f'block_{block_id}_expand_BN')(x)
        x = layers.ReLU(6., name=f'block_{block_id}_expand_relu')(x)

        # Depthwise convolution
        x = layers.DepthwiseConv2D(
            kernel_size=kernel,
            strides=strides,
            depth_multiplier=1,
            padding='same',
            use_bias=False,
            name=f'block_{block_id}_depthwise'
        )(x)
        x = layers.BatchNormalization(name=f'block_{block_id}_depthwise_BN')(x)
        x = layers.ReLU(6., name=f'block_{block_id}_depthwise_relu')(x)

        # Projection layer
        x = layers.Conv2D(
            filters,
            kernel_size=1,
            padding='same',
            use_bias=False,
            activation=None,
            name=f'block_{block_id}_project'
        )(x)
        x = layers.BatchNormalization(name=f'block_{block_id}_project_BN')(x)

        # Residual connection if input and output shapes match
        if tf.keras.backend.int_shape(inputs)[-1] == filters and strides == 1:
            x = layers.Add(name=f'block_{block_id}_add')([inputs, x])

        return x

    # Input layer
    inputs = layers.Input(shape=input_shape)

    # Initial convolution
    x = layers.Conv2D(32, kernel_size=3, strides=2, padding='same', use_bias=False, name='Conv1')(inputs)
    x = layers.BatchNormalization(name='bn_Conv1')(x)
    x = layers.ReLU(6., name='Conv1_relu')(x)

    # Depthwise separable convolution blocks
    x = _depthwise_conv_block(x, 16, block_id=1, strides=1)
    x = _depthwise_conv_block(x, 24, block_id=2, strides=2)
    x = _depthwise_conv_block(x, 24, block_id=3, strides=1)
    x = _depthwise_conv_block(x, 32, block_id=4, strides=2)
    x = _depthwise_conv_block(x, 32, block_id=5, strides=1)
    x = _depthwise_conv_block(x, 32, block_id=6, strides=1)
    x = _depthwise_conv_block(x, 64, block_id=7, strides=2)
    x = _depthwise_conv_block(x, 64, block_id=8, strides=1)
    x = _depthwise_conv_block(x, 96, block_id=9, strides=1)
    x = _depthwise_conv_block(x, 96, block_id=10, strides=1)
    x = _depthwise_conv_block(x, 160, block_id=11, strides=2)
    x = _depthwise_conv_block(x, 160, block_id=12, strides=1)
    x = _depthwise_conv_block(x, 320, block_id=13, strides=1)

    # Final convolution and pooling
    x = layers.Conv2D(1280, kernel_size=1, padding='same', use_bias=False, name='Conv_1')(x)
    x = layers.BatchNormalization(name='Conv_1_bn')(x)
    x = layers.ReLU(6., name='out_relu')(x)

    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Fully connected layers
    x = layers.Flatten()(x)
    x = layers.Dropout(0.3)(x)  # Increased dropout for regularization
    x = layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = models.Model(inputs=inputs, outputs=outputs)

    # Compile model
    initial_learning_rate = 0.001
    decay_steps = 1000
    decay_rate = 0.9

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model_mbn_build.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

# Create the model
model = mobilenet_like_model()
model.summary()

In [ ]:
# Learning rate schedule
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Model checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model_mbn_build.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

In [ ]:


history = model2.fit(train_images,
                    validation_data=val_images,
                    epochs=30,
                    callbacks=[early_stopping, checkpoint])

In [ ]:
results = model2.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])
plt.title('Training and Validation losses')
plt.xlabel('epoch')


plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Training', 'Validation'])
plt.title('Training and Validation binary_accuracy')
plt.xlabel('epoch')


In [ ]:
pred = model2.predict(test_images)
pred = np.argmax(pred, axis=1)

In [ ]:
predict_data=test_df.copy()
labels={}
for l,v in test_images.class_indices.items():
    labels.update({v:l})
predict_data['pred']=pred
predict_data['pred']=predict_data['pred'].apply(lambda x: labels[x])

In [ ]:
predict_data=predict_data.reset_index(drop=True)
predict_data.head(10)
predict_data=predict_data.reset_index(drop=True)
predict_data.head(10)

In [ ]:
predict_data[predict_data['Label']!=predict_data['pred']]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
print(f"Accuracy Score: {accuracy_score(predict_data['Label'],predict_data['pred'])}")
plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(predict_data['Label'],predict_data['pred']), annot=True, fmt='2d')

In [ ]:
print(classification_report(predict_data['Label'],predict_data['pred']))